# Module 2 Assignment

### Import Packages

In [28]:
import pandas as pd
from cassandra.cluster import Cluster

### Create a session connection to Cassandra cluster

In [29]:
clstr = Cluster()
session = clstr.connect()

### Use session to 'talk' to cassandra

In [30]:
session.execute("DROP KEYSPACE IF EXISTS m14")

In [31]:
session.execute("CREATE KEYSPACE IF NOT EXISTS m14 WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':2}")

In [32]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

m14
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


#### Create a inventory table

In [33]:
session.execute("""
CREATE TABLE IF NOT EXISTS m14.inventory ( 
    SKU UUID,
    name TEXT,
    description TEXT,
    warehouse_num INT,
    PRIMARY KEY(SKU)
);
""")

#### Create and load a DataFrame with 10 different rows of data

In [34]:
import uuid

data = {
    'SKU': [uuid.uuid4() for _ in range(10)],
    'name': ['Product A', 'Product B', 'Product C', 'Product D', 'Product E',
             'Product F', 'Product G', 'Product H', 'Product I', 'Product J'],
    'description': ['Description A', 'Description B', 'Description C', 'Description D', 'Description E',
                    'Description F', 'Description G', 'Description H', 'Description I', 'Description J'],
    'warehouse_num': [1, 2, 1, 3, 2, 3, 1, 2, 3, 4]
}

df = pd.DataFrame(data)

In [35]:
df

,SKU,name,description,warehouse_num
0,f6b61798-b2d9-47cb-a948-1fe701b6e3db,Product A,Description A,1
1,df79f01c-96ba-47cc-b38f-ed045815ae54,Product B,Description B,2
2,8f10ba90-57db-4d56-b3e2-7cb79299cb41,Product C,Description C,1
3,4fa4b391-a9a3-4d60-93a4-369a48c31791,Product D,Description D,3
4,37b70468-89fe-4652-a9fa-543d99bd816f,Product E,Description E,2
5,b6b66514-befe-4c49-a66b-7de4600a25bc,Product F,Description F,3
6,d75e19bb-2a7d-4d23-86a2-c7777c609849,Product G,Description G,1
7,7ed4b7ae-4f85-43ac-bd47-69d3f915d6e3,Product H,Description H,2
8,278a6aa0-8503-432c-8d62-6a4a2d3757a6,Product I,Description I,3
9,efe6566d-c0c0-4c3c-a02b-86b498147bed,Product J,Description J,4


In [36]:
for index, row in df.iterrows():
    session.execute(f"""
        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES (%s, %s, %s, %s)       
        """,
        (row[0], row[1], row[2], row[3])
       )

In [37]:
rows = session.execute("select (SKU, name, description, warehouse_num) from m14.inventory")
for row in rows:
    print(f"SKU={row[0][0]}, name={row[0][1]}, description={row[0][2]}, warehouse_num={row[0][3]}")

SKU=b6b66514-befe-4c49-a66b-7de4600a25bc, name=Product F, description=Description F, warehouse_num=3
SKU=4fa4b391-a9a3-4d60-93a4-369a48c31791, name=Product D, description=Description D, warehouse_num=3
SKU=37b70468-89fe-4652-a9fa-543d99bd816f, name=Product E, description=Description E, warehouse_num=2
SKU=278a6aa0-8503-432c-8d62-6a4a2d3757a6, name=Product I, description=Description I, warehouse_num=3
SKU=d75e19bb-2a7d-4d23-86a2-c7777c609849, name=Product G, description=Description G, warehouse_num=1
SKU=8f10ba90-57db-4d56-b3e2-7cb79299cb41, name=Product C, description=Description C, warehouse_num=1
SKU=f6b61798-b2d9-47cb-a948-1fe701b6e3db, name=Product A, description=Description A, warehouse_num=1
SKU=7ed4b7ae-4f85-43ac-bd47-69d3f915d6e3, name=Product H, description=Description H, warehouse_num=2
SKU=efe6566d-c0c0-4c3c-a02b-86b498147bed, name=Product J, description=Description J, warehouse_num=4
SKU=df79f01c-96ba-47cc-b38f-ed045815ae54, name=Product B, description=Description B, wareho

#### Creating an index on warehouse_num

In [38]:
session.execute("CREATE INDEX IF NOT EXISTS warehouse_num_index ON m14.inventory (warehouse_num);")

#### Searching for all products located at a given location i.e warehouse_num

In [39]:
rows = session.execute("select name from m14.inventory where warehouse_num = 1")
for row in rows:
    print(f"{row[0]}")

Product G
Product C
Product A
